Импортируем библиотеки

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from ipywidgets import interact
import ipywidgets as widgets
from ipywidgets import VBox
from scipy.interpolate import interp1d, lagrange, CubicSpline, interp1d, pchip
from scipy.interpolate import BarycentricInterpolator, Akima1DInterpolator

Создаём Pandas DataFrame из файла IBM.csv

In [9]:
file_path = 'MSFT.csv'
df = pd.read_csv(file_path)

Отрисуем график стоимости акций

In [10]:
x=np.array(np.arange(len(df['Date'])))
y=df['Volume']/np.max(df['Volume'])
x_smooth = np.linspace(x.min(), x.max(), len(df['Date']) * 2)
x_1, x_2, x_3, x_4, x_5, x_6 = np.floor([0, len(x)/5, 2*len(x)/5, 3*len(x)/5, 4*len(x)/5, len(x)])

trace1 = go.Scatter(x=x, y=y, mode="markers", name='Исходные данные')
fig1 = go.FigureWidget(layout_yaxis_range=[0,np.max(y)*1.1])
fig1.add_trace(trace1)
fig1.update_xaxes(title='Дата')
fig1.update_yaxes(title='Стоимость')
fig1

FigureWidget({
    'data': [{'mode': 'markers',
              'name': 'Исходные данные',
              'type': 'scatter',
              'uid': '3eb95699-4551-4448-8e00-afe4a69bbc4e',
              'x': array([  0,   1,   2, ..., 248, 249, 250]),
              'y': array([0.24770853, 0.28021068, 0.235998  , ..., 0.30674084, 0.17341293,
                          0.21531904])}],
    'layout': {'template': '...',
               'xaxis': {'title': {'text': 'Дата'}},
               'yaxis': {'range': [0, 1.1], 'title': {'text': 'Стоимость'}}}
})

### Линейная интерполяция

In [16]:
# Построение линейной регрессии
interp = interp1d(x, y, kind='linear')
y_linear = interp(x_smooth)
# Построение графика
trace2 = go.Scatter(
    x=x_smooth,
    y=y_linear,
    mode='markers',
    name='Линейная интерполяция'
)

fig2 = go.FigureWidget(data=[trace1, trace2])
fig2.update_layout(xaxis_title='Дата', yaxis_title='Стоимость')

def redraw_linear(**args):
    result = np.zeros_like(x, dtype=bool)
    for arg in args.values():
        result[arg[0]:arg[1]] = True

    x_changed = x[result]
    interp = interp1d(x_changed, y[x_changed], kind='linear')
    y_linear = interp(x_smooth)
    fig2["data"][1].x = x_smooth
    fig2["data"][1].y = y_linear

interact(
    redraw_linear, 
    int1=widgets.IntRangeSlider(value=[x_1, x_2], min=x_1, max=x_2, step=1,description='int1:'), 
    int2=widgets.IntRangeSlider(value=[x_2, x_3], min=x_2, max=x_3, step=1,description='int2:'),
    int3=widgets.IntRangeSlider(value=[x_3, x_4], min=x_3, max=x_4, step=1,description='int3:'), 
    int4=widgets.IntRangeSlider(value=[x_4, x_5], min=x_4, max=x_5, step=1,description='int4:'), 
    int5=widgets.IntRangeSlider(value=[x_5, x_6], min=x_5, max=x_6, step=1,description='int5:'), 
)
fig2

interactive(children=(IntRangeSlider(value=(0, 50), description='int1:', max=50), IntRangeSlider(value=(50, 10…

FigureWidget({
    'data': [{'mode': 'markers',
              'name': 'Исходные данные',
              'type': 'scatter',
              'uid': '2cd4bbb6-b867-437a-9e7b-93552e4f8342',
              'x': array([  0,   1,   2, ..., 248, 249, 250]),
              'y': array([0.24770853, 0.28021068, 0.235998  , ..., 0.30674084, 0.17341293,
                          0.21531904])},
             {'mode': 'markers',
              'name': 'Линейная интерполяция',
              'type': 'scatter',
              'uid': 'b46fca45-39a8-43f8-a4db-db2fcbf8e0c9',
              'x': array([  0.        ,   0.499002  ,   0.99800399, ..., 249.00199601,
                          249.500998  , 250.        ]),
              'y': array([0.24770853, 0.26392717, 0.28014581, ..., 0.17349657, 0.19440781,
                          0.21531904])}],
    'layout': {'template': '...', 'xaxis': {'title': {'text': 'Дата'}}, 'yaxis': {'title': {'text': 'Стоимость'}}}
})

### Интерполяция Лагранжа

In [12]:
# Построение линейной регрессии
x_l=x[0:15]
y_l=y[0:15]
interp = lagrange(x_l, y_l)
y_lagrange = interp(x_smooth[0:15])
# Построение графика
trace2 = go.Scatter(
    x=x_smooth[0:15],
    y=y_lagrange[0:15],
    mode='markers',
    name='Интерполяция лагранжа'
)

fig3 = go.FigureWidget(data=[trace1, trace2])
fig3.update_layout(xaxis_title='Дата', yaxis_title='Стоимость')

def redraw_lagrange(**args):
    result = np.zeros_like(x_l, dtype=bool)
    for arg in args.values():
        result[arg[0]:arg[1]] = True

    x_changed = x_l[result]
    interp =  lagrange(x_changed, y_l[x_changed])
    y_lagrange = interp(x_smooth)
    fig3["data"][1].y = y_lagrange

interact(
    redraw_lagrange, 
    int1=widgets.IntRangeSlider(value=[x_1, x_2], min=x_1, max=x_2, step=1,description='int1:'), 
    int2=widgets.IntRangeSlider(value=[x_2, x_3], min=x_2, max=x_3, step=1,description='int2:'),
    int3=widgets.IntRangeSlider(value=[x_3, x_4], min=x_3, max=x_4, step=1,description='int3:'), 
    int4=widgets.IntRangeSlider(value=[x_4, x_5], min=x_4, max=x_5, step=1,description='int4:'), 
    int5=widgets.IntRangeSlider(value=[x_5, x_6], min=x_5, max=x_6, step=1,description='int5:'), 
)
fig3

### Сплайн интерполяция

In [13]:
# Построение линейной регрессии
interp = CubicSpline(x, y)
y_cubic = interp(x_smooth)
# Построение графика
trace2 = go.Scatter(
    x=x_smooth,
    y=y_cubic,
    mode='markers',
    name='Сплайн интерполяция'
)

fig4 = go.FigureWidget(data=[trace1, trace2])
fig4.update_layout(xaxis_title='Дата', yaxis_title='Стоимость')

def redraw_cubic(**args):
    result = np.zeros_like(x, dtype=bool)
    for arg in args.values():
        result[arg[0]:arg[1]] = True

    x_changed = x[result]
    interp = CubicSpline(x_changed, y[x_changed])
    y_cubic = interp(x_smooth)
    fig4["data"][1].y = y_cubic

interact(
    redraw_cubic, 
    int1=widgets.IntRangeSlider(value=[x_1, x_2], min=x_1, max=x_2, step=1,description='int1:'), 
    int2=widgets.IntRangeSlider(value=[x_2, x_3], min=x_2, max=x_3, step=1,description='int2:'),
    int3=widgets.IntRangeSlider(value=[x_3, x_4], min=x_3, max=x_4, step=1,description='int3:'), 
    int4=widgets.IntRangeSlider(value=[x_4, x_5], min=x_4, max=x_5, step=1,description='int4:'), 
    int5=widgets.IntRangeSlider(value=[x_5, x_6], min=x_5, max=x_6, step=1,description='int5:'), 
)
fig4

interactive(children=(IntRangeSlider(value=(0, 50), description='int1:', max=50), IntRangeSlider(value=(50, 10…

FigureWidget({
    'data': [{'mode': 'markers',
              'name': 'Исходные данные',
              'type': 'scatter',
              'uid': 'a0949dd1-0f73-4e78-8ae9-cd5ec191adf2',
              'x': array([  0,   1,   2, ..., 248, 249, 250]),
              'y': array([0.24770853, 0.28021068, 0.235998  , ..., 0.30674084, 0.17341293,
                          0.21531904])},
             {'mode': 'markers',
              'name': 'Сплайн интерполяция',
              'type': 'scatter',
              'uid': 'c6b4579f-cc2c-4c86-a5b6-b7001fac2eb4',
              'x': array([  0.        ,   0.499002  ,   0.99800399, ..., 249.00199601,
                          249.500998  , 250.        ]),
              'y': array([0.24770853, 0.28983824, 0.28030903, ..., 0.17307661, 0.126364  ,
                          0.21531904])}],
    'layout': {'template': '...', 'xaxis': {'title': {'text': 'Дата'}}, 'yaxis': {'title': {'text': 'Стоимость'}}}
})

### Интерполяция Ньютона

In [15]:
def divided_diff(x, y):
    n = len(y)
    coef = np.zeros([n, n])
    coef[:,0] = y
    for j in range(1,n):
        for i in range(n-j):
            coef[i][j] = (coef[i+1][j-1] - coef[i][j-1]) / (x[i+j]-x[i])     
    return coef

def newton_poly(coef, x_data, x):
    n = len(x_data) - 1 
    p = coef[n]
    for k in range(1,n+1):
        p = coef[n-k] + (x -x_data[n-k])*p
    return p

a_s = divided_diff(x, y)[0, :]

y_newton = newton_poly(a_s, x, x_smooth)

trace2 = go.Scatter(
    x=x_smooth,
    y=y_newton,
    mode='markers',
    name='Интерполяция Ньютона'
)

fig5 = go.FigureWidget(data=[trace1, trace2])
fig5.update_layout(xaxis_title='Дата', yaxis_title='Стоимость')

def redraw_cubic(**args):
    result = np.zeros_like(x, dtype=bool)
    for arg in args.values():
        result[arg[0]:arg[1]] = True

    x_changed = x[result]
    y_changed = y[result]
    a_s = divided_diff(x_changed, y_changed)[0, :]
    y_newton = newton_poly(a_s, x_changed, x_smooth)
    fig5["data"][1].y = y_newton

interact(
    redraw_cubic, 
    int1=widgets.IntRangeSlider(value=[x_1, x_2], min=x_1, max=x_2, step=1,description='int1:'), 
    int2=widgets.IntRangeSlider(value=[x_2, x_3], min=x_2, max=x_3, step=1,description='int2:'),
    int3=widgets.IntRangeSlider(value=[x_3, x_4], min=x_3, max=x_4, step=1,description='int3:'), 
    int4=widgets.IntRangeSlider(value=[x_4, x_5], min=x_4, max=x_5, step=1,description='int4:'), 
    int5=widgets.IntRangeSlider(value=[x_5, x_6], min=x_5, max=x_6, step=1,description='int5:'), 
)
fig5

interactive(children=(IntRangeSlider(value=(0, 50), description='int1:', max=50), IntRangeSlider(value=(50, 10…

FigureWidget({
    'data': [{'mode': 'markers',
              'name': 'Исходные данные',
              'type': 'scatter',
              'uid': '316f50fe-5b4b-46a1-bd0d-74684916b8be',
              'x': array([  0,   1,   2, ..., 248, 249, 250]),
              'y': array([0.24770853, 0.28021068, 0.235998  , ..., 0.30674084, 0.17341293,
                          0.21531904])},
             {'mode': 'markers',
              'name': 'Интерполяция Ньютона',
              'type': 'scatter',
              'uid': '9d15b398-6839-46a7-bd01-046b6f5155c9',
              'x': array([  0.        ,   0.499002  ,   0.99800399, ..., 249.00199601,
                          249.500998  , 250.        ]),
              'y': array([2.47708532e-001, 1.51457871e+055, 7.60453940e+051, ..., 7.01066016e+108,
                          1.62799557e+109, 3.76670434e+109])}],
    'layout': {'template': '...', 'xaxis': {'title': {'text': 'Дата'}}, 'yaxis': {'title': {'text': 'Стоимость'}}}
})